In [1]:
# Importing Libraries
from pyspark import SparkContext

In [2]:
# Creating Spark Context Instance
sc = SparkContext.getOrCreate()

In [3]:
# Data Formatter
def format_input(value):
    key = value.split(" ")[0]
    frnd = value.split(" ")[1]
    return key, frnd

In [4]:
# Mapper 
def friendmap(value):
    frnds_list = value[1]
    current_frnd = value[0]
    keys = []

    for friend in frnds_list:
        key_value = ''
        if current_frnd < friend:
            key_value = current_frnd + "," +friend
        else:
            key_value = friend + "," +current_frnd
        keys.append((''.join(key_value), frnds_list))
    return keys

In [5]:
# Reading Input
Lines = sc.textFile("facebook_combined.txt", 1)

In [6]:
# Preprocessing the data
formatted_data = Lines.map(format_input).groupByKey().map(lambda x : (x[0], list(x[1])))
formatted_data.saveAsTextFile('dataset_formatted')

In [7]:
Line = formatted_data.flatMap(friendmap)

In [8]:
def get_num(x): 
    length = len(x) 
    common = [] 
    for i in range(length): 
        k = i + 1
        for j in range(k, length): 
            if x[i] == x[j] and x[i] not in common: 
                common.append(x[i]) 
    return common
result = Line.reduceByKey(lambda x, y : x+y).map(lambda x: (x[0],get_num(x[1])))

In [9]:
result.saveAsTextFile("mutual_friends_output_for_dataset")